In [1]:
%load_ext autoreload
%autoreload 2

import torch
from tokenizers import Tokenizer
import unicodedata
import time
import random
from re_sent_splitter import split_into_sentences
from tqdm import tqdm
# import nltk
# from nltk import tokenize
# nltk.download('punkt')

In [2]:
with open('../Data/english_docs_aa.txt') as f:
    docs = [line for line in f.readlines()]

In [27]:
test_docs = docs[:5000]

In [40]:
def tester(docs):
    sentences = []
    count = 0
    
    s = time.perf_counter()
    for doc in docs:
        sents = split_into_sentences(unicodedata.normalize("NFKD", doc))
        sentences.append(sents)
        count += 1
        if count % 10000 == 0:
            print(f'{count} completed')
    e = time.perf_counter() - s
    sents = [s for s in sentences if len(s.split()) > 4]
    print(round(e, 2))
    return sents

In [3]:
split_into_sentences('a bunhc of text. And some more.  And maybe 5.2% more or could be 5. 5% more.')

['a bunhc of text.',
 'And some more.',
 'And maybe 5.2% more or could be 5.5% more.']

In [4]:
tokenizer = Tokenizer.from_file('/home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Preprocessing/Tokenization/wordPiece_tokenizer_config_30500.json')

In [50]:

count = 0

start = time.perf_counter()
with open('../Data/english_docs_aa.txt') as f:
    docs = [line for line in f.readlines()]
    with open('../Data/test_sentences.txt', 'a') as w:
        for doc in docs:
            count += 1
            if count % 10000 == 0:
                print(f'{count} doc sentenceized')
            try:
                sentences = split_into_sentences(unicodedata.normalize('NFKD', doc))
                for sentence in sentences:
                    if len(sentence.split()) > 4:
                        w.write(sentence.strip())
                        w.write('\n')
                w.write('\n')
            except Exception as e:
                print(e)
                continue
end = time.perf_counter() - start
print(f'Total time: {round(end/60, 1)} minutes')

10000 doc sentenceized
20000 doc sentenceized
30000 doc sentenceized
40000 doc sentenceized
50000 doc sentenceized
60000 doc sentenceized
70000 doc sentenceized
80000 doc sentenceized
90000 doc sentenceized
Total time: 4.2 minutes


In [46]:
from transformers import BertTokenizer

In [47]:
tok = BertTokenizer(vocab_file='../Preprocessing/Tokenization/wp-vocab-30500-vocab.txt', tokenizer_file='../Preprocessing/Tokenization/wordPiece_tokenizer_config_30500.json')

In [58]:
tensors = tok.batch_encode_plus(sentences, max_length=128, padding=True, truncation=True, return_token_type_ids=False, return_tensors='pt')